In [1]:
# Import essential libraries
import requests
import cv2
import cv2.aruco as aruco
import numpy as np
import imutils
import import_ipynb
import cards_recognition
import time

# Player images
players = [
cv2.imread('winners/left.png'),
cv2.imread('winners/up.png'),
cv2.imread('winners/right.png'),
cv2.imread('winners/down.png')
]

# IP Address displayed in the IP Webcam applicaiton
url = "http://192.168.1.225:8080/shot.jpg"
#url = "http://10.227.158.141:8080/shot.jpg"

importing Jupyter notebook from cards_recognition.ipynb
importing Jupyter notebook from card_identifiers.ipynb


In [2]:
# Camera calibration values
# Henrique tele
cameraMatrix = np.array([[1078.65920, 0.00000000, 672.955927], [0.00000000, 1066.11282, 534.055095],[0.00000000, 0.00000000, 1.00000000]])
cameraDistortion = np.array([[0.13570124, -1.20889359,  0.01040627,  0.00942016,  1.56531542]])

In [3]:
trophyAxis = np.float32([[0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 0, 0],
                         [0, 0, 1.5], [0, 1, 1.5], [1, 1, 1.5], [1, 0, 1.5]])

def drawTrophy(img, rvec, tvec):
    # projection matrix points
    imgpts, _ = cv2.projectPoints(trophyAxis, rvec, tvec, cameraMatrix, cameraDistortion)
    imgpts = np.int32(imgpts).reshape(-1, 2)
    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]], -1, (0, 255, 0), 3)
    # draw pillars in blue color
    for i, j in zip(range(4), range(4, 8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]), (255), 3)
    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]], -1, (0, 0, 255), 3)
    return img


In [4]:
# winner picture vectices
winnerAxis = np.float32([[0.25, 0, 2.2], [0.25, 1, 2.2],[0.75, 1, 1.5],[0.75, 0, 1.5]]) 

# display the winner's picture
def drawWinner(img,winner,rvec,tvec):
    h, w, c = winner.shape

    # projection matrix points
    pts1, _ = cv2.projectPoints(winnerAxis, rvec, tvec, cameraMatrix, cameraDistortion)
    pts2 = np.float32([[0,0],[w,0],[w,h],[0,h]])

    # get homography matrix and warp the picture accordingly
    matrix, _ = cv2.findHomography(pts2, pts1)
    imgOut = cv2.warpPerspective(winner,matrix,(img.shape[1],img.shape[0]))

    # Merge picture with original image
    cv2.fillConvexPoly(img, pts1.astype(int),(0,0,0))
    imgOut = img + imgOut

    return imgOut

In [5]:
# get the points of the bounding box into a more convenient array
def getBoundingBoxPts(bbox):
    tl = bbox[0][0][0], bbox[0][0][1]  # top left
    tr = bbox[0][1][0], bbox[0][1][1]  # top right
    br = bbox[0][2][0], bbox[0][2][1]  # bottom right
    bl = bbox[0][3][0], bbox[0][3][1]  # bottom left
    return np.array([tl,tr,br,bl])


In [6]:
# Axis Orientations
coordinates = np.array(
    [[0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 0, 0]], dtype='float32')

# Seach for aruco markers


def findArucoMarkers(img, winner):
    # Gray image
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # needed parameters for aruco
    arucoDict = aruco.Dictionary_get(aruco.DICT_4X4_50)
    arucoParam = aruco.DetectorParameters_create()

    # Detect markers and store bounding boxes and ids
    bboxs, ids, _ = aruco.detectMarkers(
        imgGray, arucoDict, parameters=arucoParam)

    # Check if ids list is set
    if ids is not None:
        # Draw contours of detected markers
        aruco.drawDetectedMarkers(img, bboxs)
        for idx, id in enumerate(ids):
            # Check for id = 20 specificaly
            if id == 20:
                # make the points of the bounding box more convenient                
                pts = getBoundingBoxPts(bboxs[idx])

                # get the rotation and translation vectors
                _, rvec, tvec, _ = cv2.solvePnPRansac(coordinates ,pts, cameraMatrix, cameraDistortion)

                # draw axis          
                cv2.drawFrameAxes(img, cameraMatrix, cameraDistortion, rvec, tvec, 0.5)
                
                # draw trophy
                img = drawTrophy(img,rvec,tvec)
               
                # draw winner
                img = drawWinner(img,players[winner],rvec,tvec)
    return img
    


In [7]:
# Main Loop
t = time.time()
# While loop to continuously fetching data from the Url
while True:
    ##
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1280, height=720)

    # time since last frame
    deltatime = time.time() - t
    t = time.time()

    # detect, identify and process winner
    img, winner = cards_recognition.get_cards(img, deltatime)
 
    # Show round winner with aruco marker
    if winner != -1:
        img = findArucoMarkers(img, winner)
    
    # To test aruco markers without playing (0: left, 1: up, 2: right, 3: down)
    # img = findArucoMarkers(img, 0) 

    # Display Result
    cv2.imshow("Android_cam", img)

    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break
cv2.destroyAllWindows()


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Uma ligação existente foi forçada a fechar pelo anfitrião remoto', None, 10054, None))